In [8]:
import numpy as np
import pandas as pd

In [9]:
df = pd.read_csv("C:/Users/oekpom/Desktop/Datasets/Machine Learning/Lesson 10/BX-Books.csv", encoding = "ISO-8859-1")

C:\Users\oekpom\AppData\Roaming\Python\Python36\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
df.head()

,isbn,book_title,book_author,year_of_publication,publisher
0,195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,2005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,60973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271379 entries, 0 to 271378
Data columns (total 5 columns):
isbn                   271379 non-null object
book_title             271379 non-null object
book_author            271378 non-null object
year_of_publication    271379 non-null object
publisher              271377 non-null object
dtypes: object(5)
memory usage: 5.2+ MB


In [12]:
df.dtypes

isbn                   object
book_title             object
book_author            object
year_of_publication    object
publisher              object
dtype: object

In [13]:
df.isnull().sum()

isbn                   0
book_title             0
book_author            1
year_of_publication    0
publisher              2
dtype: int64

In [14]:
df.dropna(inplace = True)

In [15]:
df.isnull().sum()

isbn                   0
book_title             0
book_author            0
year_of_publication    0
publisher              0
dtype: int64

In [16]:
df_ratings = pd.read_csv("C:/Users/oekpom/Desktop/Datasets/Machine Learning/Lesson 10/BX-Book-Ratings.csv", encoding = "ISO-8859-1")

In [17]:
df_ratings.head()

,user_id,isbn,rating
0,276725,034545104X,0
1,276726,155061224,5
2,276727,446520802,0
3,276729,052165615X,3
4,276729,521795028,6


In [18]:
df_ratings.isnull().sum()

user_id    0
isbn       0
rating     0
dtype: int64

In [19]:
df_ratings.user_id.unique().shape[0]

95513

In [20]:
df_ratings.isbn.unique().shape[0]

322102

In [21]:
df_ratings.dtypes

user_id     int64
isbn       object
rating      int64
dtype: object

In [22]:
from sklearn.preprocessing import LabelEncoder

In [23]:
isbn = df_ratings.iloc[:,1]

isbn_encoder = LabelEncoder()
isbn = isbn_encoder.fit_transform(isbn)

In [24]:
userid = df_ratings.iloc[:,0]

userid_encoder = LabelEncoder()
userid = userid_encoder.fit_transform(userid)

In [25]:
df_ratings["isbn"] = pd.DataFrame(data=isbn)
df_ratings["user_id"] = pd.DataFrame(data=userid)

In [26]:
df_ratings.head()

,user_id,isbn,rating
0,94663,4511,0
1,94664,30815,5
2,94665,158458,0
3,94666,9704,3
4,94666,177084,6


In [27]:
df_ratings = df_ratings[:2000]
df_ratings.head()

,user_id,isbn,rating
0,94663,4511,0
1,94664,30815,5
2,94665,158458,0
3,94666,9704,3
4,94666,177084,6


In [28]:
from sklearn.model_selection import train_test_split
n_users = df_ratings.user_id.unique().shape[0]
n_books = df_ratings.isbn.unique().shape[0]
train_data, test_data = train_test_split(df_ratings,test_size=0.25)

In [29]:
train_data_matrix = np.zeros((n_users,n_books))
for line in train_data.itertuples():
    try:
        train_data_matrix[line[1]-1,line[2]-1] = line[3]
    except:
        continue
train_data_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [30]:
test_data_matrix = np.zeros((n_users,n_books))
for line in test_data.itertuples():
    try:
        test_data_matrix[line[1]-1,line[2]-1] = line[3]
    except:
        continue
test_data_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [31]:
from sklearn.metrics import pairwise_distances
user_similarity = pairwise_distances(train_data_matrix, metric='cosine')
movie_similarity = pairwise_distances(train_data_matrix.T, metric='cosine')
mean_user_rating = train_data_matrix.mean(axis=1)[:,np.newaxis]
ratings_diff = (train_data_matrix - mean_user_rating)
user_pred = mean_user_rating + user_similarity.dot(ratings_diff)/np.array([np.abs(user_similarity).sum(axis=1)]).T
user_pred

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])